In [1]:
import cv2
import numpy as np
import glob
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
%matplotlib inline
import PIL
import math
from keras import backend as K 
from tensorflow.keras import layers 
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.layers import Dense, Input
from sklearn.ensemble import IsolationForest


Using TensorFlow backend.


In [2]:
def SSIMLoss(y_true, y_pred):
    
    y_true = tf.convert_to_tensor(y_true)
    y_pred = tf.convert_to_tensor(y_pred)

    return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred,1.0))

In [3]:
random_state = np.random.RandomState(42)

In [4]:
def cutWorkSpace(img,border): 
    # dimensioni immagini
    img_h = 2048
    img_l = 2048
    # offset per posizionamento su area d'indagine
    offset_x = 600
    offset_y = 500
    
    #taglio per garantire la ricerca sull'area di interesse, evitando rumori dati dallo sfondo
    crop_img = img[offset_y:img_h-offset_y, offset_x:img_l-offset_x]
    
    # Convert the img to grayscale
    gray = cv2.cvtColor(crop_img,cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    high_thresh, thresh_im = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    #cv2.imwrite('provaT.jpg', thresh_im)
    lowThresh = 0.5*high_thresh
    
    
    # Apply edge detection method on the image
    #edges = cv2.Canny(blurred,20,150,apertureSize = 3)
    edges = cv2.Canny(blurred,18,150,apertureSize = 3)
    #plt.imshow(edges)
    #plt.show()
    #cv2.imwrite('results_img/'+str(count_img)+'ed.jpg', edges)
    #cv2.imwrite('canny.jpg', edges)
    # This returns an array of r and theta values
    lines = cv2.HoughLines(edges,1,np.pi/180, 100)
    #cv2.imwrite('Risultati/resFS'+name+'c.jpg', edges)
    # The below for loop runs till r and theta values 
    # are in the range of the 2d array
    i = 0
    x_first = 0
    x_second = 0
    count = 0
    delta = 35
    thetaFinal = 0
    while count < 2:
        for r,theta in lines[i]:
            # Stores the value of cos(theta) in a
            a = np.cos(theta)

            # Stores the value of sin(theta) in b
            b = np.sin(theta)

            # x0 stores the value rcos(theta)
            x0 = a*r

            # y0 stores the value rsin(theta)
            y0 = b*r

            #ti define line lenght
            const = 3000
            # x1 stores the rounded off value of (rcos(theta)-1000sin(theta))
            x1 = int(x0 + const*(-b))

            # y1 stores the rounded off value of (rsin(theta)+1000cos(theta))
            y1 = int(y0 + const*(a))

            # x2 stores the rounded off value of (rcos(theta)+1000sin(theta))
            x2 = int(x0 - const*(-b))

            # y2 stores the rounded off value of (rsin(theta)-1000cos(theta))
            y2 = int(y0 - const*(a))

            if count == 0 :
                if theta != 0:
                    thetaFinal = 180-math.degrees(theta)
                x_first = x0
                #cv2.line(img,(x1+offset_x,y1+offset_y), (x2+offset_x,y2+offset_y), (0,0,255),2)
                count += 1
            else:
                if abs(x_first-x0) > delta :
                    x_second = x0
                    #cv2.line(img,(x1+offset_x,y1+offset_y), (x2+offset_x,y2+offset_y), (0,0,255),2)
                    count += 1
            i += 1
    # All the changes made in the input image are finally
    # written on a new image houghlines.jpg
    #cv2.imwrite('Risultati/res'+name+'.jpg', img)
    (h, w) = img.shape[:2]
    (cX, cY) = (w // 2, h // 2)
    # rotate our image by Theta degrees around the center of the image
    M = cv2.getRotationMatrix2D((cX, cY), -thetaFinal, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h))
    scale_percent = 100 # percent of original size

  
    if x_first < x_second:
        
        median = (x_second+x_first)//2
        temp = rotated[0:img_h-1, int(offset_x+median-border):int(offset_x+median+border)]
        width = int(temp.shape[1] * scale_percent / 100)
        height = int(temp.shape[0] * scale_percent / 100)
        dim = (width, height)
        temp = cv2.resize(temp, dim, interpolation = cv2.INTER_AREA)        
        gray = cv2.cvtColor(temp,cv2.COLOR_BGR2GRAY)
        return  gray
    else:
        median = (x_first+x_second)//2.
        temp = rotated[0:img_h-1, int(offset_x+median-border):int(offset_x+median+border)]        
        #blurred = cv2.GaussianBlur(temp, (3,3), 0)
        width = int(temp.shape[1] * scale_percent / 100)
        height = int(temp.shape[0] * scale_percent / 100)
        dim = (width, height)
        temp = cv2.resize(temp, dim, interpolation = cv2.INTER_AREA)
        gray = cv2.cvtColor(temp,cv2.COLOR_BGR2GRAY)
        return  gray
    

In [5]:
def divideImage(img,pcs):
    results = []
    lung = len(img) // pcs
    for i in range(pcs):
        results.append(img[(lung*i):(lung*(i+1)-1)][:])
    final_res = []
    for res in results:
        final_res.append(cv2.resize(res, (rows,col), interpolation = cv2.INTER_AREA))
    return final_res

In [16]:
#pathBuoni = '/Users/leonardoaggio/Desktop/Dataset/S606C/Cam3@00004/*.tif'
pathBuoni = '/Users/leonardoaggio/Desktop/Dataset/S606C/Buoni/*.tif'
pathScarti = '/Users/leonardoaggio/Desktop/Dataset/Dataset/SectionScarti/*.tif'


rows = 32
col = 32

pezziXimmagine = 17
border = 60
#Nclusters = 10

results = []
img_train = []
list_paths = []


list_paths_scarti = glob.glob(pathScarti) 
list_paths_buoni = glob.glob(pathBuoni)
results += [1] * (len(list_paths_buoni) * pezziXimmagine)
results += [-1] * len(list_paths_scarti)
list_paths += list_paths_scarti
list_paths += list_paths_buoni

count = 0
for img_path in list_paths_buoni:
    img = cv2.imread(img_path)
    #try:
    img_train +=  divideImage(cutWorkSpace(img,border),pezziXimmagine)
    count += 1
#except:
    #print('errore lettura')

for img_path in list_paths_scarti:
    img = cv2.imread(img_path)
   # try:
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (rows,col), interpolation = cv2.INTER_AREA)
    img_train.append(img)
#except:
    #print('errore lettura')        

x_train = np.array(img_train)


x_train = x_train/255.
x_temp = np.empty((len(img_train), rows,col,1))
x_train = x_train.astype('float32')
for i in range(len(img_train)):
    x_temp[i]=(np.expand_dims(x_train[i], axis=2))
x_train = x_temp

x_train = x_train.astype('float32')

In [17]:
print(len(img_train))

2363


In [18]:
optimizer = tf.keras.optimizers.Adam(lr = 0.0005)
autoencoder_model = tf.keras.models.load_model('/Users/leonardoaggio/Desktop/Dataset/Programmi/Autoencoder/newFormatVersion3', compile=False)
autoencoder_model.compile(optimizer=optimizer, loss=SSIMLoss)

get_6th_layer_output = K.function([autoencoder_model.layers[0].input],
                                  [autoencoder_model.layers[6].output])

In [19]:
layer_output_train = get_6th_layer_output([x_train])[0]

In [74]:
contamination_N = len(list_paths_scarti)/ len(results) * 1.05

model=IsolationForest(n_estimators=100,verbose = 1,max_samples='auto',contamination=contamination_N,random_state=random_state)

model.fit(layer_output_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s finished


IsolationForest(contamination=0.03776978417266187,
                random_state=RandomState(MT19937) at 0x7FF5643048D0, verbose=1)

In [75]:
train_predict = model.predict(layer_output_train)

In [76]:
print(train_predict)

[ 1  1  1 ...  1 -1 -1]


In [77]:
j = 0
for i in range (len(results)):
    if train_predict[i] != results[i]:
       # print(train_predict[i])
       # print(results[i])
        j += 1
print(j)

63


In [89]:
path = '/Users/leonardoaggio/Desktop/Dataset/S606C/ScartiTotali/*.tif'

list_paths = []
limite = 4
scarto = 0
list_paths = glob.glob(path) 

for img_path in list_paths:
    img = cv2.imread(img_path)
    #try:
    imgs =  divideImage(cutWorkSpace(img,border),pezziXimmagine) 

    x_test = np.array(imgs)

    x_test = x_test/255.
    x_temp = np.empty((len(x_test), rows,col,1))
    x_test = x_test.astype('float32')
    for i in range(len(x_test)):
        x_temp[i]=(np.expand_dims(x_test[i], axis=2))
    x_test = x_temp
    x_test = x_test.astype('float32')
    layer_output_test = get_6th_layer_output([x_test])[0]
    test_predict = model.predict(layer_output_test)
    count = 0
    for i in test_predict:
        if i == -1:
            count +=1
    if count > limite:
        scarto+= 1
    print(count)

12
13
9
4
4
15
5
4
3


In [87]:
scarto

5

In [88]:
len(list_paths)

9